**Agricultural drought indicators & geospatial data handling in Python**

In this exercise we use data of drought-related indicators on agricultural fields, namely
- SPEI : Standarized Precipitation-Evaporation Index
- SMI : Soil Moisture Index
- LST/NDVI : Land Surface Temperature / Normalized Difference Vegetation Index
- AZL : Ackerzahl (soil quality)
- TWI : Topographic Wetness Index
- NFK : plant available water (*nutzbare Feldkapazität*)

First we import the module "os" which contains some functions related to the operating system. For example we might want to check the current working directory ("getcwd"). Modules in Python are typically written in object-oriented coding style and functions are called by [module name].[function name] e.g.
*os.getcwd()*

In the next step we import the module "geopandas" that allows us to work with geospatial data. Renaming this module to the abbreviation "gpd" is a convention - not strictly necessary, though. Same is true for seaborn, a library that creates nice plots (similar to ggplot in R)

In [ ]:
import os
import geopandas as gpd
import seaborn as sns

os.getcwd()

The module os contains a function *.listdir* to list the files in the directory. There should be a folder called 'data'. Change the code here yourself and repeat the command for this subfolder by adding the folder name in the brackets: *os.listdir('data')*

In [ ]:
os.listdir()

Using geopandas we can read a geopackage (.gpkg) file like the subset of agricultural fields around Cottbus, and directly visualize it in an interactive map by the function *.explore* - this can take 15-30 seconds

In [ ]:
data_cottbus = gpd.read_file("data/subset_cottbus.gpkg")
data_cottbus.explore()

Apparently the dataset has many attributes (columns) and contains information on crops for several years. It makes sense to check the actual data in tabular format (attribute table in QGIS). This information can be obtained by simply typing the name of the data object.

To access only a few rows, we can use the function *.head()* (first few columns), *.tail()* (last few columns) or *.sample()* (random sample). A number can be written within the brackets of the function call, to specify how many rows to display.

Change the code yourself to display different numbers of rows from the top, bottom, or random sample of the data

In [ ]:
data_cottbus.head(10)

The unique values contained in a column may be displayed by the function *.unique()*

In [ ]:
data_cottbus.year.unique()

To select rows by a criterion, the function *.loc[]* can be used. Note that *.loc[]* requires square brackets in which the selection criteria for the dimensions of the data are formulated. The next code block demonstrates a query to select rows of only the year 2022, but containing all columns.

Change this code yourself to select a few rows and columns of interest

In [ ]:
data_cottbus.loc[data_cottbus.year == 2022,]

A sequence of Python commands can be run at once (piped). With this knowlege we can easily display a selection of the data by a single line of code. The *.explore()* function can be provided a column name as optional agrument, to display the data colored by the values in the column. When using a categorical attribute like "crop", *.explore()* automatically uses a nice categorical color schema.

In [ ]:
data_cottbus.loc[data_cottbus.year == 2022,].explore("crop")

**Spatial intersection**


We can add another layer like the shapes of the Landkreise of Brandenburg and visualize both at the same time

In [ ]:
brandenburg = gpd.read_file("data/brandenburg_landkreise_id_25833.gpkg")
mapplot = brandenburg.explore(name="Landkreise", style_kwds=dict(fillOpacity=.1))
mapplot = data_cottbus.explore(m=mapplot, color="red", name="Fields")
mapplot

Note that the data is automatically projected to a matching coordinate reference system (CRS)! This is very handy when exploring data. However, it can be dangerous when working with the data. Make sure you always check the CRS when working with geospatial files. In Python you can do this with the geopandas command *.crs* - no () brackets here

In [ ]:
brandenburg.crs

In [ ]:
data_cottbus.crs

Transformation of the CRS is very simple as well. We can either provide the EPSG code as an argument, or the CRS of another object.

In [ ]:
data_cottbus = data_cottbus.to_crs(brandenburg.crs)

# check
if data_cottbus.crs == brandenburg.crs:
    print("CRS identical")
else:
    raise ValueError("CRS check failed")

Load the random subset of 10.000 fields across Brandenburg ("subset_random.gpkg") and select data from a single year. Then merge this with the shapes of the Landkreise and plot the density of LST/NDVI-anomalies per Landkreis.

In [ ]:
data_random = gpd.read_file("data/subset_random.gpkg")
data_2022 = data_random.loc[data_random.year == 2022, ]
data_random.explore("crop")
#data_2022.explore("crop")

Use the function gpd.overlay() to intersect the layers

In [ ]:
intersected = gpd.overlay(data_2022.to_crs(brandenburg.crs), brandenburg, how="intersection", keep_geom_type=False)
intersected.head()

**Descriptive Statistics**

Summary statistics of the attributes (columns) can be obtained by the function *.describe()*

The histogram or kernel density of individual columns can be plotted by *.plot(kind="density")*

In [ ]:
data_cottbus.describe()

In [ ]:
data_cottbus.groupby("year").lstndvi_anom.plot(kind="density", legend=True)

Compute a correlation matrix using the random samples across Brandenburg. The area of Cottbus is too small to capture variability in the features (the meteorological indicator has a grid resolution of 10 km)

In [ ]:
data_random.corr(numeric_only=True).round(1)

The *clustermap* function from the seaborn module nicely visualizes such a correlation matrix

In [ ]:
sns.clustermap(data_random.corr(numeric_only=True).round(1), cmap="coolwarm")

**Statistical distributions per Landkreis**

Remove Potsdam (too few samples), group by krs_name, select the column lstndvi_anom, and plot the density

In [ ]:
intersected.loc[intersected.krs_name != "Potsdam",].groupby("krs_name").lstndvi_anom.plot(kind="density", legend=True)

Compute statistical moments per Landkreis

In [ ]:
intersected.loc[intersected.krs_name != "Potsdam",].groupby("krs_name").lstndvi_anom.mean()

**Threshold features and relative affected area**

Select a single crop type and only the columns "lstndvi_anom", "area_iacs", and "krs_name"
Compute affected area > thresholds per Landkreis.

Let's start by defining a threshold value of 0.2 and calling the new column "th_02" respectively.

In [ ]:
# repeating the intersection with data from all years
intersected_all = gpd.overlay(data_random.to_crs(brandenburg.crs), brandenburg, how="intersection", keep_geom_type=False)
wheat = intersected_all.loc[intersected_all.crop == "winter_wheat", ["crop", "year", "krs_name", "area_iacs", "lstndvi_anom"]]

# define a new column that indicates whether a threshold values is exceeded (True or False)
wheat["th_02"] = wheat.lstndvi_anom > 0.2

# 'True' is treated as 1 while 'False' is treated as 0
# we can thus multiply the new column with the area to calculate the total affected area
# above the specified threshold per Landkreis

wheat["affected_area"] = wheat["area_iacs"].multiply(wheat["th_02"])
wheat

Print the sum of affected area per Landkreis

In [ ]:
wheat.groupby(["krs_name", "year"]).affected_area.sum().head(40)

Create a barplot of affected area > threhold per year. (keep in mind that the area is only based on a random subset of the agricultural fields)

In [ ]:
wheat.groupby(["year"]).affected_area.sum().plot(kind="bar", title="Wheat - affected area of LST/NDVI-anom. > 0.2")

Divide by the absolute area per crop to obtain the relative affected area (fraction). For the sake of readibility, first create a new object for the yearly sums, and then add a column called relative_affected_area

In [ ]:
yearly_sums = wheat[["year", "area_iacs", "affected_area"]].groupby(["year"]).sum()
yearly_sums["relative_affected_area"] = yearly_sums["affected_area"].divide(yearly_sums["area_iacs"])
yearly_sums

In [ ]:
yearly_sums.relative_affected_area.plot(kind="bar", color = "gold", 
                                        title="Wheat - relative affected area with LST/NDVI-anom. > 0.2")

**Now it's your turn:**

You can add new code cells and/or markdown (text) cells below and conduct your analysis from scratch, or copy and change code cells from above. Please have a look at the tasks in the exercise and homework sheets. The easiest way to export a figure from this online environment is by taking a screenshot. Please also provide the code cell used to create the figure in an Appendix section of your report.